In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

!pip install yake
import yake

!pip install rake-nltk -q
from rake_nltk import Rake, Metric

notes = pd.read_csv("../input/nbme-score-clinical-patient-notes/patient_notes.csv")


import re
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string

In [ ]:
def process_tweet(tweet):
    '''
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    '''
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
    
    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english):
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    note = ""
    for word in tweets_clean:
        note += word + " "
        
    return note


def merge(notes):
    notes.pn_history.str.lower()
    case = ["" for i in range(10)]

    for i, j in notes.iterrows():
        case[j.case_num] += j.pn_history + " "
    
    for i in range(len(case)):
        case[i] = process_tweet(case[i])
    
    return case


def yake_learn(cases, case_num):
    kw_extractor = yake.KeywordExtractor()
    keywords = kw_extractor.extract_keywords(cases[case_num])
    return keywords


def rake_learn(case):
    r = Rake(min_length=2, max_length=5, include_repeated_phrases=False, ranking_metric=Metric.WORD_DEGREE)
    r.extract_keywords_from_text(case)
    rake_results = r.get_ranked_phrases_with_scores()
    return rake_results

In [ ]:
cases = merge(notes)

yake_results = []
for i in range(10):
    sub_arr = []
    sub_arr.append(yake_learn(cases, i))
    yake_results.append(sub_arr)
    del sub_arr


rake_results = []
for i in range(10):
    sub_arr = []
    sub_arr.append(rake_learn(cases[i]))
    rake_results.append(sub_arr)
    del sub_arr

In [ ]:
cases = merge(notes)
"""
with open('rake', 'w') as f:
    for i in rake_results:
        f.write(i)
"""

In [ ]:
with open('yake.csv', 'w') as f:
    for i in range(10):
        for j in yake_learn(cases, i):
            f.write(j[0])
            f.write(',')
        f.write('\n')

In [ ]:
with open('rake.csv', 'w') as f:
    for i in range(10):
        for j in rake_learn(cases[i]):
            f.write(j[1])
            f.write(',')
        f.write('\n')